In [1]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F 

In [2]:
# hyperparameters
batch_size = 32 
block_size = 8 
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2

device = 'cuda' if torch.cuda.is_available() else "cpu"
eval_iters = 200

In [3]:
torch.manual_seed(1337)

In [5]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [6]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [11]:
# get the unique characters in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [44]:
# create a mapping from characters to integers and vice-versa

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # string to list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # integers to string

In [13]:
# train and test splits

data = torch.tensor(encode(text), dtype=torch.long)

In [14]:
n = int(.9*len(data)) # reserve 90% for training and 10% for validation
train_data = data[:n]
val_data = data[n:]

In [17]:
# data loading 

def get_batch(split):
    # create a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [82]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)

            # focus only on the last time step 
            logits = logits[:, -1, :] # (B,C)
            # apply softmax to get the probabilites
            probs = F.softmax(logits, dim=-1) # (B,C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append the sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # (B,T+1)
        return idx

In [92]:
model = BigramLanguageModel()

model = model.to(device)

In [79]:
# out = model.generate(torch.tensor([[7]]),5)

In [80]:
# [decode(x.tolist()) for x in out]

["-'VlDa"]

In [93]:
@torch.no_grad()
def evaluate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [94]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [98]:
for it in range(10000):
    if (it % eval_iters) == 0:
        losses = evaluate_loss()
        print(f"step {it}: train loss {losses['train']:.4f} val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)

print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


step 0: train loss 2.4503 val loss 2.4970
step 200: train loss 2.4568 val loss 2.4964
step 400: train loss 2.4574 val loss 2.4930
step 600: train loss 2.4595 val loss 2.4883
step 800: train loss 2.4634 val loss 2.4908
step 1000: train loss 2.4528 val loss 2.4903
step 1200: train loss 2.4582 val loss 2.5023
step 1400: train loss 2.4617 val loss 2.4911
step 1600: train loss 2.4571 val loss 2.4900
step 1800: train loss 2.4492 val loss 2.4977
step 2000: train loss 2.4589 val loss 2.4981
step 2200: train loss 2.4548 val loss 2.4946
step 2400: train loss 2.4549 val loss 2.4920
step 2600: train loss 2.4595 val loss 2.4892
step 2800: train loss 2.4640 val loss 2.4887
step 3000: train loss 2.4578 val loss 2.4844
step 3200: train loss 2.4570 val loss 2.4940
step 3400: train loss 2.4535 val loss 2.4813
step 3600: train loss 2.4548 val loss 2.4915
step 3800: train loss 2.4567 val loss 2.4937
step 4000: train loss 2.4574 val loss 2.4930
step 4200: train loss 2.4591 val loss 2.4920
step 4400: train 